
http://www.etax.nat.gov.tw/etwmain/front/ETW144W12

## get company info

http://www.etax.nat.gov.tw/etwmain/front/ETW144GetPulicInfo?ban=24755908

{"cpOwner":"張家齊","cpHsnCd":"A","cpName":"木刻思股份有限公司","cpAddr":"臺北市信義區興雅里基隆路１段１５５號８樓之８"}

##submitted form

對 http://www.etax.nat.gov.tw/etwmain/front/ETW144W12 POST

hsnCd是台北市
dstCd是信義區國稅局
idf:為非境內居住者
idf2為自動補扣並補繳

```
toConfirm:
isConfirm:
payTypeName:152 租賃
uniformCode:24755908
withholdingUnitName:木刻思股份有限公司
withholdingPeople:張家齊
withholdingAddress:臺北市信義區興雅里基隆路１段１５５號８樓之８
hsnCd:A
dstCd:0300
idf:2
varYear:104
varMonth:04
varYear1:104
varMonth1:04
varDay1:27
pwamt1:1,984
pwamt2:1,984
rate:20
withholdingAmount:396
idf1:N
btnSubmit:確認送出
```

onsubmit="return checkField();


getPublicInfo
etw.ETW144.utils.js:339 
``` {javascript}
        
        function getPublicInfo(ban, cpNameTag, cpAddrTag, cpOwnerTag, cpHsnCdTag,
                dstTag, sp) {
            // var $j = jQuery.noConflict();
            $j.ajax({
                url : '/etwmain/front/ETW144GetPulicInfo?ban=' + ban,
                type : 'get',
                dataType : 'json',
                error : function(xhr) {
                    // alert('讀取類別發生錯誤');
                },
                success : function(data) {
                    if (!$j.isEmptyObject(data)) {
                        // data["cpHsnCd"] = 'E'; //JF
                        $j("#" + cpNameTag).val(data.cpName);
                        if (cpOwnerTag) {
                            $j("#" + cpOwnerTag).val(data.cpOwner);
                        }
                        $j("#" + cpAddrTag).val(data.cpAddr);
                        $j("#" + cpHsnCdTag).children().each(function() {
                            if ($j(this).val() == data.cpHsnCd) {
                                $j(this).attr("selected", "true");
                                // 連動稽徵單位
                                if (dstTag) {
                                    getDst(data.cpHsnCd, dstTag, sp);
                                }
                            }
                        });
                    }
                }
            });
        }
```
checkField 
ETW144W12:1573 
```
function checkField(){
document.getElementById("divDownloadPdf").style.display = 'none';
document.getElementById('isConfirm').value = "";
document.getElementById('toConfirm').value = "";
  var errormsg = '';
  var p_cat = $F('p_cat'); //繳款類別
  var hsn_cd = $F('hsn_cd'); //縣市
  var ban = $F('ban'); //統一編號
  var dst_cd = $F('dst_cd'); //稽徵單位
  
  var idx = $('dst_cd').selectedIndex;
  var dst_cd_text = $('dst_cd').options[idx].text; //稽徵單位代碼全文字
  var be_name = $F('be_name'); //扣繳單位名稱
  var be_add = $F('be_add'); //扣繳單位地址
  var be_owner = $F('be_owner');  //扣繳義務人
  var owner_cat = $F('owner_cat'); //所得人身分別
  var ear_yyy = $F('ear_yyy'); //所屬年
  var ear_mm = $F('ear_mm'); //所屬月
  var gd_yyy = $F('gd_yyy'); //給付年
  var gd_mm = $F('gd_mm'); //給付月
  var gd_dd = $F('gd_dd'); //給付日
  var pw_rate = $F('pw_rate'); //扣繳率
  var g_amt = $F('g_amt'); //給付所得總額
  var a_amt = changeToNormalFormat($F('a_amt')); //應扣繳所得額
  var pw_amt = changeToNormalFormat($F('pw_amt')); //應扣繳稅額

  //99.12.02加千分位
  var pw_amt_n = changeToMoneyFormat($F('pw_amt')); //應扣繳稅額
//  alert('pw_amt_n='+pw_amt_n);

  var count_pw_amt = 0 ;

  //必填欄位
  if(p_cat=='-1'){
    errormsg = errormsg + '請選擇「繳款類別」\n';
  }
  if(hsn_cd=='-1' ){
    errormsg = errormsg + '請選擇「縣市」\n';
  }
  if(dst_cd=='-1' ){
    errormsg = errormsg + '請選擇「稽徵單位」\n';
  }
  
  /**2015/02/06 檢查縣市跟稽徵單位有無對應到 By Pablo**/
  if (hsn_cd != '-1' && dst_cd != '-1') {
      if (!checkDstMapping(hsn_cd, dst_cd)) {
          errormsg = errormsg + '請重新選擇「縣市」及「稽徵單位」\n';
      }
  } 
  
  if(isNullorSpace(ban) ){
    errormsg = errormsg + '請完整填寫「統一編號」\n';
  }else{
    if(!isCompanyBan(ban) ){
      errormsg = errormsg + '請輸入正確之「統一編號」\n';
    }
  }
  if(owner_cat=='-1' ){
    errormsg = errormsg + '請選擇「所得人身分別」\n';
  }
  if(isNullorSpace(ear_yyy) || isNullorSpace(ear_mm) ){
    errormsg = errormsg + '請完整填寫「所屬年月」\n';
  }else{
    if(ear_mm < 1 || ear_mm > 12){
      errormsg = errormsg + '「所屬年月」之月份請介於1~12\n';
    }
  }
  //if(isNullorSpace(gd_yyy) || isNullorSpace(gd_mm) || isNullorSpace(gd_dd) ){
 if(isNullorSpace(gd_dd)) {
	gd_dd = "01";
 }
 if(isNullorSpace(gd_yyy) || isNullorSpace(gd_mm)){
    errormsg = errormsg + '請完整填寫「給付日期」\n';
  }else{
      var tmp_date = gd_yyy+gd_mm+gd_dd;
      var gdc_date = Number(tmp_date)+19110000;
      if(!isDate(gdc_date.toString())){errormsg = errormsg + '「給付日期」請輸入正確日期格式\n';}
  }
  var pw_amt_count = $('pw_amt').value;

  if(pw_amt_count.length>13){
    errormsg = errormsg + '「應扣繳稅額」大於10位 (十億), 無法使用此種方式繳納\n';
  }

  if(isNullorSpace(pw_amt)|| pw_amt=='0'){
    errormsg = errormsg + '請完整填寫「應扣繳稅額」\n';
  }

  //長度檢核
  if(be_name.Blength()>0){
    if(be_name.Blength()>52){
      errormsg = errormsg + '「扣繳單位名稱」之欄位內容長度請勿超過26個中文字\n';
    }
  }
  /*if(be_owner.Blength()>0){
    if(be_owner.Blength()>12){
      errormsg = errormsg + '「扣繳義務人」之欄位內容長度請勿超過6個中文字\n';
    }
  }*/
  if(be_add.Blength()>0){
    if(be_add.Blength()>80){
      errormsg = errormsg + '「扣繳單位地址」之欄位內容長度請勿超過40個中文字\n';
    }
  }
  if(getCheckedRadioValue('idf1') == '') {
	errormsg = errormsg + '您尚未填寫是否自動補扣並補繳\n';
  }

  var auto_amt = document.getElementsByName('auto_amt');
  var auto_amt_v='';

  for (var i=0; i<auto_amt.length; i++){
      if (auto_amt[i].checked){
          auto_amt_v = auto_amt[i].value;
          break;
      }
  }

  count_pw_amt = cal_pw_amt(a_amt, pw_rate);

  //navywang 2011-05-24 add:
  be_name = checksymbol(be_name);//檢查特殊符號並替換
  be_add = checksymbol(be_add);//檢查特殊符號並替換
  be_owner = checksymbol(be_owner);//檢查特殊符號並替換
  //navywang add end

  	if(errormsg != ''){
      alert(errormsg);
	  return false;
    } else {
        if(!confirm('※產製繳款書前，請確認：\n\n 1.是否已安裝PDF檔案閱讀軟體?\n\n 2.產製繳款書功能僅相容於Micorsoft Internet Exploer 瀏覽器6.x、7.x、8.x、9.x版本(不建議使用於5.x(含)以下之版本)\n\n 3.印表機是否已正確安裝並能正常使用? (建議使用雷射印表機)\n\n請按「確定」繼續產製，或按「取消」取消產製。')){
        	return false;
        }

        var _year = parseInt(ear_yyy, 10) + 1911;
    	var now = new Date();
        if (now.getFullYear() != new Number(_year)) {
           return confirm("你所輸入之「所屬年度」非屬今年度，請確認是否產製繳款書");
        }
        return true;
    }

}
```
